In [1]:

import numpy as np
import pandas as pd
from seeds import known_seeds
from utils import save_solution
from evaluation import get_actual_demand


def get_my_solution(d):
    # This is just a placeholder.
    return [{}]


seeds = known_seeds('training')

demand = pd.read_csv('./data/demand.csv')
for seed in seeds:
    # SET THE RANDOM SEED
    np.random.seed(seed)

    # GET THE DEMAND
    actual_demand = get_actual_demand(demand)

    # CALL YOUR APPROACH HERE
    solution = get_my_solution(actual_demand)



In [2]:
df_servers = pd.read_csv('./data/servers.csv')
df_servers

,server_generation,server_type,release_time,purchase_price,slots_size,energy_consumption,capacity,life_expectancy,cost_of_moving,average_maintenance_fee
0,CPU.S1,CPU,"[1,60]",15000,2,400,60,96,1000,288
1,CPU.S2,CPU,"[37,96]",16000,2,460,75,96,1000,308
2,CPU.S3,CPU,"[73,132]",19500,2,800,120,96,1000,375
3,CPU.S4,CPU,"[109,168]",22000,2,920,160,96,1000,423
4,GPU.S1,GPU,"[1,72]",120000,4,3000,8,96,1000,2310
5,GPU.S2,GPU,"[49,120]",140000,4,3000,8,96,1000,2695
6,GPU.S3,GPU,"[97,168]",160000,4,4200,8,96,1000,3080


In [3]:
df_selling_price = pd.read_csv('./data/selling_prices.csv')
df_selling_price

,server_generation,latency_sensitivity,selling_price
0,CPU.S1,low,10.0
1,CPU.S2,low,10.0
2,CPU.S3,low,11.0
3,CPU.S4,low,12.0
4,GPU.S1,low,1500.0
5,GPU.S2,low,1600.0
6,GPU.S3,low,2150.0
7,CPU.S1,medium,15.0
8,CPU.S2,medium,15.0
9,CPU.S3,medium,16.5


In [4]:
df_datacenters = pd.read_csv('./data/datacenters.csv')
df_datacenters

,datacenter_id,cost_of_energy,latency_sensitivity,slots_capacity
0,DC1,0.25,low,25245
1,DC2,0.35,medium,15300
2,DC3,0.65,high,7020
3,DC4,0.75,high,8280


# Calculate the total cost when buy a server

In [5]:
df_results = []

for i_server in range(len(df_servers)):
    for i_datacenter in range(len(df_datacenters)):
        server_generation = df_servers['server_generation'][i_server]
        datacenter_id = df_datacenters['datacenter_id'][i_datacenter]
        cost_of_energy = df_datacenters['cost_of_energy'][i_datacenter]
        latency_sensitivity = df_datacenters['latency_sensitivity'][i_datacenter]

        selling_price = df_selling_price[np.logical_and(df_selling_price['server_generation'] == server_generation, 
                                                        df_selling_price['latency_sensitivity'] == latency_sensitivity)]['selling_price'].iloc[0]


        life_expectancy = df_servers['life_expectancy'][i_server]    
        purchase_price = df_servers['purchase_price'][i_server]
        energy_consumption_per_day = df_servers['energy_consumption'][i_server]
        energy_cost_per_day = energy_consumption_per_day * cost_of_energy
        average_mantainance_cost_per_day = df_servers['average_maintenance_fee'][i_server]
        server_capacity = df_servers['capacity'][i_server]

        
        for average_U in np.linspace(0.0, 1.0, 21):
            total_cost = purchase_price
            total_profit = 0
            list_objective_per_day = []
            for i in range(1, life_expectancy+1):
                U = average_U
                L = i / life_expectancy
                R = server_capacity * U * selling_price
                C = average_mantainance_cost_per_day * (1 + 1.5 * i / life_expectancy * np.log2(1.5 * i / life_expectancy))
                P = R - C
                list_objective_per_day.append([U, L, P])

                total_cost += energy_cost_per_day
                total_profit += P
            
            list_objective_per_day = np.asarray(list_objective_per_day)
            objective_per_day = list_objective_per_day[:,0] * list_objective_per_day[:,1] * list_objective_per_day[:,2]
            total_objective = np.sum(objective_per_day)

            df_results.append([server_generation, datacenter_id, average_U,
                                 total_cost, total_profit, total_objective])
                               

df_results = pd.DataFrame(df_results, columns=['server_generation', 'datacenter_id', 'average_U', 'total_cost', 'total_profit', 'total_objective'])
df_results


,server_generation,datacenter_id,average_U,total_cost,total_profit,total_objective
0,CPU.S1,DC1,0.00,24600.0,-2.495036e+04,0.000000
1,CPU.S1,DC1,0.05,24600.0,-2.207036e+04,-703.945828
2,CPU.S1,DC1,0.10,24600.0,-1.919036e+04,-1262.391655
3,CPU.S1,DC1,0.15,24600.0,-1.631036e+04,-1675.337483
4,CPU.S1,DC1,0.20,24600.0,-1.343036e+04,-1942.783310
...,...,...,...,...,...,...
583,GPU.S3,DC4,0.80,462400.0,1.392050e+06,537562.713949
584,GPU.S3,DC4,0.85,462400.0,1.495730e+06,615683.383571
585,GPU.S3,DC4,0.90,462400.0,1.599410e+06,699042.053193
586,GPU.S3,DC4,0.95,462400.0,1.703090e+06,787638.722815


In [6]:
# df_results group by server_generation and datacenter_id, calculate minimum value of average_U that make total_objective positive
df_results = df_results[df_results['total_objective'] > 0]

In [7]:
df_results

,server_generation,datacenter_id,average_U,total_cost,total_profit,total_objective
11,CPU.S1,DC1,0.55,24600.0,6.729644e+03,259.095897
12,CPU.S1,DC1,0.60,24600.0,9.609644e+03,1155.650069
13,CPU.S1,DC1,0.65,24600.0,1.248964e+04,2197.704242
14,CPU.S1,DC1,0.70,24600.0,1.536964e+04,3385.258414
15,CPU.S1,DC1,0.75,24600.0,1.824964e+04,4718.312586
...,...,...,...,...,...,...
583,GPU.S3,DC4,0.80,462400.0,1.392050e+06,537562.713949
584,GPU.S3,DC4,0.85,462400.0,1.495730e+06,615683.383571
585,GPU.S3,DC4,0.90,462400.0,1.599410e+06,699042.053193
586,GPU.S3,DC4,0.95,462400.0,1.703090e+06,787638.722815


In [8]:
df_results[np.abs(df_results['average_U'] - 0.95) < 1e-3]

,server_generation,datacenter_id,average_U,total_cost,total_profit,total_objective
19,CPU.S1,DC1,0.95,24600.0,2.976964e+04,11505.529276
40,CPU.S1,DC2,0.95,28440.0,5.712964e+04,24636.904276
61,CPU.S1,DC3,0.95,39960.0,1.118496e+05,50899.654276
82,CPU.S1,DC4,0.95,43800.0,1.118496e+05,50899.654276
103,CPU.S2,DC1,0.95,27040.0,4.171698e+04,17046.409781
124,CPU.S2,DC2,0.95,31456.0,7.591698e+04,33460.628531
145,CPU.S2,DC3,0.95,44704.0,1.443170e+05,66289.066031
166,CPU.S2,DC4,0.95,49120.0,1.443170e+05,66289.066031
187,CPU.S3,DC1,0.95,38700.0,8.789656e+04,38562.918849
208,CPU.S3,DC2,0.95,46380.0,1.480886e+05,67451.943849


In [9]:
df_results[np.logical_and(df_results['server_generation'] == 'CPU.S1', df_results['datacenter_id'] == 'DC1')]

,server_generation,datacenter_id,average_U,total_cost,total_profit,total_objective
11,CPU.S1,DC1,0.55,24600.0,6729.643561,259.095897
12,CPU.S1,DC1,0.60,24600.0,9609.643561,1155.650069
13,CPU.S1,DC1,0.65,24600.0,12489.643561,2197.704242
14,CPU.S1,DC1,0.70,24600.0,15369.643561,3385.258414
15,CPU.S1,DC1,0.75,24600.0,18249.643561,4718.312586
16,CPU.S1,DC1,0.80,24600.0,21129.643561,6196.866759
17,CPU.S1,DC1,0.85,24600.0,24009.643561,7820.920931
18,CPU.S1,DC1,0.90,24600.0,26889.643561,9590.475104
19,CPU.S1,DC1,0.95,24600.0,29769.643561,11505.529276
20,CPU.S1,DC1,1.00,24600.0,32649.643561,13566.083449


In [10]:
df_results.groupby(['server_generation', 'datacenter_id'])['average_U'].min().reset_index()

,server_generation,datacenter_id,average_U
0,CPU.S1,DC1,0.55
1,CPU.S1,DC2,0.40
2,CPU.S1,DC3,0.25
3,CPU.S1,DC4,0.25
4,CPU.S2,DC1,0.50
5,CPU.S2,DC2,0.35
6,CPU.S2,DC3,0.20
7,CPU.S2,DC4,0.20
8,CPU.S3,DC1,0.35
9,CPU.S3,DC2,0.25


# Single server, real run

In [11]:

from utils import (load_problem_data,
                   load_solution)
from evaluation import evaluation_function, get_evaluation
import pandas as pd

demand, datacenters, servers, selling_prices = load_problem_data(path='./data')

In [12]:


from utils import (load_problem_data,
                   load_solution)
from evaluation import evaluation_function, get_known, get_evaluation
import pandas as pd

demand, datacenters, servers, selling_prices = load_problem_data(path='./data_test')
df_results = []

for i_server in range(len(df_servers)):
    for i_datacenter in range(len(df_datacenters)):
        datacenter_id = df_datacenters['datacenter_id'][i_datacenter]
        server_generation = df_servers['server_generation'][i_server]
        latency_sensitivity = df_datacenters['latency_sensitivity'][i_datacenter]
        server_capacity = df_servers['capacity'][i_server]

        solution = [{"time_step": 1, "datacenter_id": datacenter_id, "server_generation": server_generation, "server_id": "7ee8a1de-b4b8-4fce-9bd6-19fdf8c1e409", "action": "buy"}]
        solution = pd.DataFrame(solution)
        # LOAD PROBLEM DATA

        for average_U in np.linspace(0.0, 1.0, 21)[1:]:
            demand = pd.DataFrame({
                "time_step": list(range(1,168+1)),
                "server_generation": [server_generation]*168,
                "high": [0]*168,
                "medium": [0]*168,
                "low": [0]*168
            })
            demand[latency_sensitivity] = server_capacity * average_U
                
            # EVALUATE THE SOLUTION
            score = get_evaluation(solution, 
                                        demand,
                                        datacenters,
                                        servers,
                                        selling_prices,
                                        time_steps=100, 
                                        verbose=False,
                                        random_demand=False)
            
            df_results.append([server_generation, datacenter_id, average_U, score])

df_results = pd.DataFrame(df_results, columns=['server_generation', 'datacenter_id', 'average_U', 'score'])
df_results[df_results['score'] > 0].groupby(['server_generation', 'datacenter_id'])['average_U'].min().reset_index()
# df_results.join(df_servers[['server_generation', 'slots_size']], on='server_generation', how='left')
df_results = pd.DataFrame.join(df_results.set_index('server_generation'),
                  df_servers[['server_generation', 'slots_size']].set_index('server_generation'), 
                  on='server_generation', how='left').reset_index()
df_results['score_per_slot'] = df_results['score'] / df_results['slots_size']

df_results = pd.DataFrame.join(df_results.set_index('datacenter_id'),
                  df_datacenters[['datacenter_id', 'latency_sensitivity']].set_index('datacenter_id'), 
                  on='datacenter_id', how='left').reset_index()

df_results = df_results.sort_values(by=['score_per_slot', 'average_U'], ascending=False)
df_results.to_csv("df_single_server_results.csv", index=False)
df_results
